<a href="https://www.kaggle.com/code/leonliur/rwanda-co2-prediction-lb-9?scriptVersionId=149364745" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

# Progress & versions 🚀

| Changes  | CV Score LGBM | CV Score XGB | CV Score RF |
|----------|---------------|--------------|----------|
| Baseline | 19.92164              | 24.86644             |
| Removed features > 0.5 NaN         |               |24.61487              ||
| Imputed rest of NaN with mean         |               | 24.75321             ||
| Implemented scalar |  | 24.34700 ||
| Covid flag | | 23.36592||
| sin cos weeks | | 23.69442||
| is zero | | 24.47380 ||
| 8-features | | 19.58470|
| Clustered | | 15.70909 | | 
| Dist_to_max | | 15.50617 | |
| PCA | | 14.81864 | 12.52867|
| Max Emission Week Station | | 11.50131 |

# Importing Libraries 📚

In [1]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import seaborn as sns
import datetime as dt
import haversine as hs

from sklearn.preprocessing import OrdinalEncoder, StandardScaler
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
from tqdm import tqdm
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor, plot_importance
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [2]:
GPU = False

In [3]:
df_og = pd.read_csv("../input/rwanda-kfolds/train_folds.csv")
df_test_og = pd.read_csv("/kaggle/input/playground-series-s3e20/test.csv")
sample_submission = pd.read_csv("/kaggle/input/playground-series-s3e20/sample_submission.csv")

# Cleaning & Preprocessing ⚙️

Since I only ended up using only the indexing features (lon, lat, week, year, etc...), I did not need to do much preprocessing and cleaning

In [4]:
def cleaning(df):
    return df.drop(["ID_LAT_LON_YEAR_WEEK"], axis=1)

In [5]:
def feature_preprocessing(df, drop_na_threshold):
#     # truncate columns with < 0.5 NA
#     missing_ratios = df.isnull().mean()
#     columns_to_drop = missing_ratios[missing_ratios > drop_na_threshold].index
#     df = df.drop(columns_to_drop, axis=1)
    
#     # impute the rest with mean
#     df = df.fillna(df.mean())
    

    # scalar is causing problems
#     sc = StandardScaler()
#     for i in df.columns:
#         if i not in ["emission", "covid_flag", "week_no", "year", "latitude", "longitude", "kfold"]:
#             df[i] = sc.fit_transform(df[i].values.reshape(-1,1))
    return df

In [6]:
def feature_preprocessing_inf(df, drop_na_threshold, scaler):
    # truncate columns with < 0.5 NA
#     missing_ratios = df.isnull().mean()
#     columns_to_drop = missing_ratios[missing_ratios > drop_na_threshold].index
#     df = df.drop(columns_to_drop, axis=1)
    
    # impute the rest with mean
#     df = df.fillna(df.mean())
    
#     sc = StandardScaler()
#     for i in df.columns:
#         if i not in ["emission", "covid_flag", "week_no", "year", "latitude", "longitude", "kfold"]:
#             df[i] = sc.fit_transform(df[i].values.reshape(-1,1))
    return df

# Feature Engineering 🔎

Here are the additional features I added
- cartesian rotation: these are inspired by this [medium post](https://bmanikan.medium.com/feature-engineering-all-i-learned-about-geo-spatial-features-649871d16796), where it is shown that rotating the longitude, latitude, and computing polar coordinates help
- month: the month of the recorded emission
- PCA: principal component analysis on the longitude and latitude

In [7]:
def rotation(df):
    df['rot_45_x'] = (0.707 * df['latitude']) + (0.707 * df['longitude'])
    df['rot_45_y'] = (0.707 * df['longitude']) + (0.707 * df['latitude'])
    df['rot_30_x'] = (0.866 * df['latitude']) + (0.5 * df['longitude'])
    df['rot_30_y'] = (0.866 * df['longitude']) + (0.5 * df['latitude'])
    df['r'] = np.sqrt(df['longitude']**2 + df['latitude']**2)
    df['phi'] = np.arctan2(df['latitude'], df['longitude'])
    return df

def get_month(row):
    date = dt.datetime.strptime(f'{int(row.year)}-{int(row.week_no)+1}-1', "%Y-%W-%w")
    return date.month

def find_distance(row, max_emissions):
    return hs.haversine((max_emissions[int(row.kmeans_group)][0], max_emissions[int(row.kmeans_group)][1]),(row.latitude, row.longitude))

def pca(df):
    coordinates = df[['latitude','longitude']].values
    pca_obj = PCA()
    pca_coordinates = pca_obj.fit_transform(coordinates)
    
    df['pca_x'] = pca_coordinates[:, 0]
    df['pca_y'] = pca_coordinates[:, 1]
    return df

More features!
- week_sin and week_cos are **cyclical** time features, this way week 51 can be seen as close to week 1
- covid flag flags covid months
- cluster: KMeans clustering was performed on the coordinates to group recorded areas into five groups near each other
- dist_to_max: the distance between the recorded location and the maximum CO2 location in the same cluster
- emission_zero: there are some entries with zero emission across the time frame
- stationID: the location where the data was recorded
- max_emission_week_station: **this is an important feature**: inspired by [this notebook](https://www.kaggle.com/code/danbraswell/no-ml-public-lb-23-02231/notebook), we can see that the maximum emission for any location for any week across three years give a good clue at the final emission. Therefore I have incorporated this feature as part of my feature engineering

In [8]:
N_CLUSTERS = 5

def feature_engineering(df):
    # cos and sin to week
    df["week_sin"] = np.sin(df["week_no"] / 52 * 2 * np.pi)
    df["week_cos"] = np.cos(df["week_no"] / 52 * 2 * np.pi)
    
    # covid flag
    df["covid_flag"] = np.where((df["year"] == 2020) & (df["week_no"] >= 11) & (df["week_no"] <= 40), 1, 0)
    
    df = rotation(df)
    
    # cluster K means: elbow point at 5
    cluster_df = df.groupby(by=['latitude', 'longitude'], as_index=False)['emission'].mean()
    model = KMeans(n_clusters=N_CLUSTERS, n_init='auto')
    cluster_df['kmeans_group'] = model.fit_predict(cluster_df)
    
    # calculate distance to most in emission
    idx = cluster_df.groupby("kmeans_group")["emission"].transform(max) == cluster_df["emission"]
    max_emission_indices = cluster_df.loc[idx]
    max_emissions = []
    for cluster_no in range(N_CLUSTERS):
        lon, lat = max_emission_indices.loc[max_emission_indices.kmeans_group == cluster_no, ["latitude", "longitude"]].values[0]
        max_emissions.append((lon, lat))
    cluster_df["dist_to_max"] = cluster_df.apply(lambda row : find_distance(row, max_emissions), axis=1) 
    
    # there are some 0 emissions we need to avoid
    df["emission_zero"] = np.where(df["emission"] == 0, 1, 0)
    
    # month column
    df["month"] = df.apply(lambda row : get_month(row), axis=1)
    
    # pca
    df = pca(df)
    
    # adding station ID and max emission across three years
    unique_stations = set(zip(df["latitude"], df["longitude"]))
    station_mapping = {tuple(station): index + 1 for index, station in enumerate(unique_stations)}
    df["station_id"] = df.apply(lambda row : station_mapping[(row['latitude'], row['longitude'])], axis=1)
    max_emissions = df.groupby(by=["station_id", "week_no"])["emission"].max().reset_index().rename(columns={"emission":"max_emission_week_station"})

    df = df.merge(max_emissions, on=['station_id', 'week_no'], how='left')
    df = df.merge(cluster_df[["latitude", "longitude", "kmeans_group", "dist_to_max"]], on=["latitude", "longitude"])
    
    return df

In [9]:
def feature_engineering_test(df, df_test):
    # cos and sin to week
    df["week_sin"] = np.sin(df["week_no"] / 52 * 2 * np.pi)
    df_test["week_sin"] = np.sin(df_test["week_no"] / 52 * 2 * np.pi)
    
    df["week_cos"] = np.cos(df["week_no"] / 52 * 2 * np.pi)
    df_test["week_cos"] = np.cos(df_test["week_no"] / 52 * 2 * np.pi)
    
    # covid flag
    df["covid_flag"] = np.where((df["year"] == 2020) & (df["week_no"] >= 11) & (df["week_no"] <= 40), 1, 0)
    df_test["covid_flag"] = np.where((df_test["year"] == 2020) & (df_test["week_no"] >= 11) & (df_test["week_no"] <= 40), 1, 0)
    
    df = rotation(df)
    df_test = rotation(df_test)
    
    # cluster K means: elbow point at 5
    cluster_df = df.groupby(by=['latitude', 'longitude'], as_index=False)['emission'].mean()
    model = KMeans(n_clusters=N_CLUSTERS, n_init='auto')
    cluster_df['kmeans_group'] = model.fit_predict(cluster_df)
    
    # calculate distance to most in emission
    idx = cluster_df.groupby("kmeans_group")["emission"].transform(max) == cluster_df["emission"]
    max_emission_indices = cluster_df.loc[idx]
    max_emissions = []
    for cluster_no in range(N_CLUSTERS):
        lon, lat = max_emission_indices.loc[max_emission_indices.kmeans_group == cluster_no, ["latitude", "longitude"]].values[0]
        max_emissions.append((lon, lat))
    cluster_df["dist_to_max"] = cluster_df.apply(lambda row : find_distance(row, max_emissions), axis=1) 
    
    # there are some 0 emissions we need to avoid
    cluster_df["emission_zero"] = np.where(cluster_df["emission"] == 0, 1, 0)
    
    # month column
    df["month"] = df.apply(lambda row : get_month(row), axis=1)
    df_test["month"] = df.apply(lambda row : get_month(row), axis=1)
    
    # PCA
    df = pca(df)
    df_test = pca(df_test)
    
    # adding station ID and max emission across three years
    unique_stations = set(zip(df["latitude"], df["longitude"]))
    station_mapping = {tuple(station): index + 1 for index, station in enumerate(unique_stations)}
    df["station_id"] = df.apply(lambda row : station_mapping[(row['latitude'], row['longitude'])], axis=1)
    df_test["station_id"] = df_test.apply(lambda row : station_mapping[(row['latitude'], row['longitude'])], axis=1)    
    max_emissions = df.groupby(by=["station_id", "week_no"])["emission"].max().reset_index().rename(columns={"emission":"max_emission_week_station"})
    
    # merging max emission station data
    df = df.merge(max_emissions, on=['station_id', 'week_no'], how='left')
    df_test = df_test.merge(max_emissions, on=['station_id', 'week_no'], how='left')
    
    # merging distance to max emission data
    df = df.merge(cluster_df[["latitude", "longitude", "kmeans_group", "emission_zero", "dist_to_max"]], on=["latitude", "longitude"])
    df_test = df_test.merge(cluster_df[["latitude", "longitude", "kmeans_group", "emission_zero", "dist_to_max"]], on=["latitude", "longitude"])
    
    return df,df_test

# Model Training & Inference 🪁

In [10]:
def go_kfold(df, features, df_test=None, weights=(1,1,1), coef=1, INFERENCE=False):
    coefficients = [x / sum(weights) for x in weights]
    
    cv_scores_xgb = []
    cv_scores_lgbm = []
    cv_scores_rf = []
    cv_scores_assemble = []
    
    if INFERENCE:
        final_predictions_xgb = []
        final_predictions_lgbm = []
        final_predictions_rf = []
        final_predictions_assemble = []
        X_test = df_test[features]

    for fold in range(5):
        X_train = df[df.kfold != fold]
        X_valid = df[df.kfold == fold].reset_index(drop=True)

        y_train = X_train.emission
        y_valid = X_valid.emission

        X_train = X_train[features]
        X_valid = X_valid[features]

        print("--")
        
        preds_valid_assemble = np.zeros(len(X_valid))
        if INFERENCE:
            test_preds_assemble = np.zeros(len(X_test))

        if weights[0] != 0:
            if GPU:
                model_xgb = XGBRegressor(random_state=fold, n_jobs=4, tree_method="gpu_hist", gpu_id=0, predictor="gpu_predictor")
            else:
                model_xgb = XGBRegressor(random_state=fold, n_jobs=4)
            model_xgb.fit(X_train, y_train)
            preds_valid_xgb = model_xgb.predict(X_valid) * coef
            preds_valid_assemble += preds_valid_xgb * coefficients[0]
            cv_score_xgb = mean_squared_error(y_valid, preds_valid_xgb, squared=False)
            cv_scores_xgb.append(cv_score_xgb)

            print(fold,"RMSE for XGB:", cv_score_xgb)
            
            if INFERENCE:
                test_preds_xgb = model_xgb.predict(X_test) * coef
                final_predictions_xgb.append(test_preds_xgb)
                test_preds_assemble += test_preds_xgb * coefficients[0]

        if weights[1] != 0:
            model_lgbm = LGBMRegressor(n_estimators = 1000, max_depth = 15, learning_rate = 0.01, num_leaves = 105)
            model_lgbm.fit(X_train, y_train)
            preds_valid_lgbm = model_lgbm.predict(X_valid) * coef
            preds_valid_assemble += preds_valid_lgbm * coefficients[1]
            cv_score_lgbm = mean_squared_error(y_valid, preds_valid_lgbm, squared=False)
            cv_scores_lgbm.append(cv_score_lgbm)

            print(fold,"RMSE for LGBM:", cv_score_lgbm)
            
            if INFERENCE:
                test_preds_lgbm = model_lgbm.predict(X_test) * coef
                final_predictions_lgbm.append(test_preds_lgbm)
                test_preds_assemble += test_preds_lgbm * coefficients[1]
        
        if weights[2] != 0:
            model_rf = RandomForestRegressor(n_estimators=100, random_state=42)
            model_rf.fit(X_train, y_train)
            preds_valid_rf = model_rf.predict(X_valid) * coef
            preds_valid_assemble += preds_valid_rf * coefficients[2]
            cv_score_rf = mean_squared_error(y_valid, preds_valid_rf, squared=False)
            cv_scores_rf.append(cv_score_rf)

            print(fold,"RMSE for RF:", cv_score_rf)
            
            if INFERENCE:
                test_preds_rf = model_rf.predict(X_test) * coef
                final_predictions_rf.append(test_preds_rf)
                test_preds_assemble += test_preds_rf * coefficients[2]
        
        # get CV score for assembled
        cv_score_assemble = mean_squared_error(y_valid, preds_valid_assemble, squared=False)
        cv_scores_assemble.append(cv_score_assemble)

        print(fold, "RMSE for assemble: ", cv_score_assemble)
        
        if INFERENCE:
            final_predictions_assemble.append(test_preds_assemble)
        
    print("--")
    
    if weights[0]:
        print("Mean RMSE for XGB:", np.mean(cv_scores_xgb), "| XGB std RMSE", np.std(cv_scores_xgb))
    if weights[1]:
        print("Mean RMSE for LGBM:", np.mean(cv_scores_lgbm), "| LGBM std RMSE", np.std(cv_scores_lgbm))    
    if weights[2]:
        print("Mean RMSE for RF:", np.mean(cv_scores_rf), "| RF std RMSE", np.std(cv_scores_rf))
    print("Mean RMSE for Assemble:", np.mean(cv_scores_assemble), "| Assemble std RMSE", np.std(cv_scores_assemble))
    
    if INFERENCE:
        xgb_predictions = np.zeros(df_test.shape[0]) if weights[0] == 0 else np.mean(np.column_stack(final_predictions_xgb), axis=1)
        lgbm_predictions = np.zeros(df_test.shape[0]) if weights[1] == 0 else np.mean(np.column_stack(final_predictions_lgbm), axis=1)
        rf_predictions = np.zeros(df_test.shape[0]) if weights[2] == 0 else np.mean(np.column_stack(final_predictions_rf), axis=1)

        sample_submission.emission = np.mean(np.column_stack(final_predictions_assemble), axis=1)
        sample_submission.to_csv("submission.csv", index=False)
        print("--\nKFOLD INFERENCE DONE")

In [11]:
def inference_whole_data(df, df_test, features, weights=(1,1,1), coef=1):
    coefficients = [x / sum(weights) for x in weights]

    final_predictions_xgb = []
    final_predictions_lgbm = []
    final_predictions_rf = []
    X_test = df_test[features]

    X_train = df[features]
    y_train = df.emission

    if weights[0] != 0:
        if GPU:
            model_xgb = XGBRegressor(random_state=42, n_jobs=4, tree_method="gpu_hist", gpu_id=0, predictor="gpu_predictor")
        else:
            model_xgb = XGBRegressor(random_state=42, n_jobs=4)
        model_xgb.fit(X_train, y_train)
        final_predictions_xgb.append(model_xgb.predict(X_test))

    if weights[1] != 0:
        model_lgbm = LGBMRegressor(n_estimators = 1000, max_depth = 15, learning_rate = 0.01, num_leaves = 105)
        model_lgbm.fit(X_train, y_train)
        final_predictions_lgbm.append(model_lgbm.predict(X_test))

    if weights[2] != 0:
        model_rf = RandomForestRegressor(n_estimators=100, random_state=42)
        model_rf.fit(X_train, y_train)
        final_predictions_rf.append(model_rf.predict(X_test))

    xgb_predictions =  np.zeros(df_test.shape[0]) if weights[0] == 0 else np.mean(np.column_stack(final_predictions_xgb), axis=1)
    lgbm_predictions = np.zeros(df_test.shape[0]) if weights[1] == 0 else np.mean(np.column_stack(final_predictions_lgbm), axis=1)
    rf_predictions = np.zeros(df_test.shape[0]) if weights[2] == 0 else np.mean(np.column_stack(final_predictions_rf), axis=1)
                    
    sample_submission.emission = coefficients[0] * xgb_predictions + coefficients[1] * lgbm_predictions + coefficients[2] * rf_predictions
    sample_submission.emission = sample_submission.emission * coef
    sample_submission.to_csv("submission.csv", index=False)
    print("--\nINFERENCE W/ WHOLE DATA DONE")    

In [12]:
INFERENCE = True
vital_features = ["ID_LAT_LON_YEAR_WEEK", "week_no", "year", "latitude", "longitude", "emission", "kfold"]

if INFERENCE:
    df = df_og.copy()[vital_features]
    df = cleaning(df)
    df = feature_preprocessing(df, 0.5)
    df_test = df_test_og.copy()
    df_test = cleaning(df_test)
    df_test = feature_preprocessing(df_test, 0.5)
    
    df, df_test = feature_engineering_test(df, df_test)
    features = [x for x in df.columns if x not in ('emission', 'kfold')]
    go_kfold(df, features, weights=(0, 0, 1), coef=1.07, df_test=df_test, INFERENCE=True)
else:
    df = df_og.copy()[vital_features]
    df = cleaning(df)
    df = feature_preprocessing(df, 0.5)

    df = feature_engineering(df)
    features = [x for x in df.columns if x not in ('emission', 'kfold')]
    go_kfold(df, features, weights=(0, 0, 1), coef=1.07, INFERENCE=False)

--
0 RMSE for RF: 16.712992437117336
0 RMSE for assemble:  16.712992437117336
--
1 RMSE for RF: 19.64012233093327
1 RMSE for assemble:  19.64012233093327
--
2 RMSE for RF: 27.348100665208765
2 RMSE for assemble:  27.348100665208765
--
3 RMSE for RF: 13.808304245200288
3 RMSE for assemble:  13.808304245200288
--
4 RMSE for RF: 13.805210729175922
4 RMSE for assemble:  13.805210729175922
--
Mean RMSE for RF: 18.262946081527115 | RF std RMSE 5.030988827113296
Mean RMSE for Assemble: 18.262946081527115 | Assemble std RMSE 5.030988827113296
--
KFOLD INFERENCE DONE


# That's it! 🧨

I think there is definitely room for this notebook to perform better, however, this is also a learning process for me. Please give me some feedback on how to improve!